In [79]:
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import Generate_word2vec_features
import os
import sys
import RunModels
sys.path.append( os.path.join(".."))
from UtilityFunctions import CommonHelpers,PreprocessHelpers,FeatureEngineering
import numpy as np
import pandas as pd
import re

In [11]:
gen = Generate_word2vec_features.Word2Vec_Features()
gen.load_pickles()
train_vec =np.nan_to_num( gen.book_vectors)
titles= gen.titles
categories= gen.categories


In [32]:
book_pickle_path="../data/preprocessed/books_english_preprocessed.pickle"
name_pickle_path = "../data/preprocessed/names_english.pickle"

In [33]:
list_of_words = CommonHelpers.load_pickle(book_pickle_path)
book_names =  CommonHelpers.load_pickle(name_pickle_path)

In [36]:
def get_number_id(file_name):
    regex = r"([a-z_A-Z]+)(\d+)"
    match = re.search(regex, file_name)
    if match!=None:
        return match.group(2)

In [44]:
df = pd.read_csv("../data/pg_catalog.csv")
df["category"] = df["LoCC"].str[:1]
category_list_train = [int(get_number_id(each)) for each in book_names]
df[df["Text#"]==27943]

/home/sharath/anaconda3/envs/fall21/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Text#,Type,Issued,Title,Language,Authors,Subjects,LoCC,Bookshelves,category
27872,27943,Text,2009-01-31,"The Home Medical Library, Volume 1 (of 6)",en,"Winslow, Kenelm, 1863- [Editor]","Medicine; Medicine, Popular",RC,NaN,R


In [46]:
selected_books = df[df["Text#"].isin(category_list_train)]
selected_books[selected_books["Text#"]==27943]

,Text#,Type,Issued,Title,Language,Authors,Subjects,LoCC,Bookshelves,category
27872,27943,Text,2009-01-31,"The Home Medical Library, Volume 1 (of 6)",en,"Winslow, Kenelm, 1863- [Editor]","Medicine; Medicine, Popular",RC,NaN,R


In [55]:




categories=[]
titles=[]
for each in category_list_train:
    categories.append(selected_books[selected_books["Text#"]==each]["category"].values[0])
    titles.append(selected_books[selected_books["Text#"]==each]["Title"].values[0])



In [47]:
number = 420
print(list_of_words[number][:100])
print(">>>>>>>>>>>>>>>.")
print(book_names[number],categories[number],titles[number],category_list_train[number])

['home', 'medical', 'library', 'kenelm', 'winslow', 'formerly', 'assistant', 'professor', 'comparative', 'therapeutic', 'harvard', 'university', 'late', 'surgeon', 'newton', 'hospital', 'fellow', 'massachusetts', 'medical', 'society', 'etc', 'cooeperation', 'many', 'medical', 'advising', 'editor', 'special', 'contributor', 'six', 'volume', 'first', 'aid', 'family', 'medicine', 'nose', 'throat', 'lung', 'eye', 'ear', 'stomach', 'bowel', 'tumor', 'skin', 'disease', 'rheumatism', 'germ', 'disease', 'nervous', 'disease', 'insanity', 'sexual', 'hygiene', 'woman', 'child', 'heart', 'blood', 'digestion', 'personal', 'hygiene', 'indoor', 'exercise', 'diet', 'conduct', 'long', 'life', 'practical', 'kitchen', 'science', 'nervousness', 'outdoor', 'life', 'nurse', 'patient', 'camping', 'comfort', 'sanitation', 'household', 'pure', 'water', 'supply', 'pure', 'food', 'stable', 'kennel', 'new', 'york', 'review', 'review', 'company', 'medical', 'advising', 'editor', 'managing', 'editor', 'albert', 'wa

In [57]:
print(len(train_vec),len(set(categories)),categories[420])

1400 21 R


In [58]:
forest = RandomForestClassifier(n_estimators = 100)

In [59]:
model = RunModels.Run_Model(forest,train_vec,categories)
model.run_model()
result = model.get_predictions()

In [61]:
matrix,accuracy = model.get_metrics()

In [62]:
accuracy

0.6678571428571428

In [100]:
names = [
    "Nearest Neighbors",
    "RBF SVM",
    # "Gaussian Process",
 

    "RF esti = 1000",
    "RF max depth =15 ",
    "RF no max features",
    "rf just estimators",
    "Neural Net",
    "Naive Bayes",
]
classifiers = [
    KNeighborsClassifier(3),
    SVC(gamma=2, C=1),
    # GaussianProcessClassifier(1.0 * RBF(1.0)),


    RandomForestClassifier(max_depth=20, n_estimators=1000, max_features=1),
    RandomForestClassifier(max_depth=15, n_estimators=400, max_features=1),
    RandomForestClassifier(max_depth=15, n_estimators=400),
    RandomForestClassifier( n_estimators=400),


    MLPClassifier(alpha=0.05, max_iter=10000),
    GaussianNB(),
]

In [101]:
for name, clf in zip(names, classifiers):
    model = RunModels.Run_Model(clf,train_vec,categories)
    model.run_model()
    result = model.get_predictions()
    matrix,accuracy = model.get_metrics()
    print(name,accuracy)

Nearest Neighbors 0.5678571428571428
RBF SVM 0.6
RF esti = 1000 0.6785714285714286
RF max depth =15  0.65
RF no max features 0.6607142857142857
rf just estimators 0.7
Neural Net 0.7
Naive Bayes 0.5357142857142857


In [71]:
matrix,accuracy = model.get_metrics()

In [72]:
accuracy

0.16428571428571428

TypeError: predict_proba() missing 1 required positional argument: 'X'